# LIBRARIES

In [1]:
import pandas as pd
import numpy  as np
import time
import matplotlib.pyplot as plt

from pkg.constants    import *
from pkg.evaluation   import Evaluation
from pkg.utils        import *
from pkg.algorithims  import *
from pkg.neighborhood import *



from pkg.tabu_list import TabuList

from itertools import permutations
from random    import shuffle



# SEED PRODUCTION

In [2]:
# Semillas adri
# seeds = np.array([12345, 54321, 12221, 22211, 22212])

# semillas preestablecidas
seeds = np.array([7054, 1354, 23503, 11268, 58283])
print(seeds)

[ 7054  1354 23503 11268 58283]


# EVALUATION OBJECT

In [3]:
ev = Evaluation()

# IMPLEMENATION OF ALGORITHIMS

## LOCAL SEARCH
first best neighbor

In [ ]:
granularity = 1

for i in range(SEEDS):

    np.random.seed(seeds[i])

    ls_sol, ls_sol_cost, calls = local_search(
        granularity = granularity,
        evaluation  = ev
    )
    
    print(ls_sol, round(ls_sol_cost,4), calls)

  

## SIMULATED ANNEALING

In [ ]:
# parametros a ajustar:
slots = 1

# k_max: maximo numero iteraciones: en el entorno de 80
# condicion de parada
k_max = 80

# L(T): condicion enfriamiento. Numero fijo para cada iteracion
# Se enfria la temperatura al final de una iteracion
# La iteracion finaliza al visitar todos los vecinos
L = 20


# Esquema de enfriamiento: Cauchy
# tk = t0/(1+k)

# mu: tanto por uno [0,1], indica la probabilidad de aceptar una solucion peor que la inicial
mu = 0.2

# fi: probabilidad de aceptar una solucion un mu por 1 peor que la inicial
fi = 0.2

# Temperatura inicial
t0 = mu * greedy_sol_cost / (- np.log(fi))

# Temperatura final
tf = t0 / (1 + k_max)



In [ ]:
for i in range(SEEDS):
    np.random.seed(seeds[i])
    sa_solution, sa_cost = simulated_an(t0,L,tf,slots,ev)
    print(sa_solution, round(sa_cost, 4))



##  TABU SEARCH

In [4]:
# externo
reboots = 4
total_iterations = 40

total_neighbors = 20
tenure = 4

slots = 4 # esto por que????


In [7]:
for seed in seeds:
    np.random.seed(seed)
    ts_sol, ts_sol_cost = tabu_search(
        tenure           = tenure,
        reboots          = reboots,
        total_iterations = total_iterations,
        slots            = slots,
        total_neighbors  = total_neighbors,
        evaluation       = ev
    )
    b = ts_sol
    print(ts_sol, round(ts_sol_cost, 4))

[22.  4.  6. 23.  6. 18. 20. 19.  2. 17. 15.  8.  8.  9. 15. 23.] 215.0
[21. 14. 19. 15. 18.  5. 11.  7.  9. 18. 15. 24. 11. 12.  9. 12.] 409.5282
[2.0e+001 2.8e+001 3.0e+000 1.3e+001 2.7e+001 2.5e-323 8.0e+000 2.6e+001
 2.9e+001 4.4e-323 6.0e+000 5.4e-323 5.9e-323 2.3e+001 7.0e+000 2.9e+001] 219.0
[10. 11. 18. 22. 17. 15. 13. 12. 10. 16. 15. 24.  4.  6. 12. 14.] 393.0349
[ 2.  1. 27. 22.  5. 27.  7.  5. 30. 19.  1. 15. 12. 25.  4. 15.] 217.0
[10.  6. 19. 28.  2.  7.  2. 25.  1. 22. 25. 11. 20. 11.  8. 19.] 216.0
[26. 13. 19.  8.  2. 17. 24.  1. 17.  3.  2. 26. 17.  2. 16. 25.] 218.0
[17. 11. 14. 14. 17. 11. 11.  9. 10. 17. 18. 27.  6. 14. 11. 13.] 396.0466
[0.0e+000 3.0e+001 9.9e-324 1.2e+001 2.0e-323 2.1e+001 3.0e+000 1.8e+001
 6.0e+000 2.6e+001 2.0e+001 2.7e+001 2.4e+001 2.8e+001 6.9e-323 3.0e+000] 218.0
[13. 12. 18. 18. 16. 15. 12.  7. 10. 18. 18. 25.  4. 14.  9. 11.] 388.136
[1.5e+001 1.4e+001 2.4e+001 1.6e+001 2.0e-323 2.9e+001 2.6e+001 3.5e-323
 4.0e-323 1.0e+001 4.9e-323 2.4e+0